### dependency 설치(매 pull마다 실행하기)

In [ ]:
%pip install --upgrade pip
%pip install -r requirements.txt

# llama index를 사용해 gemini api 불러오기

### png 불러오기(tesseract를 사용한 OCR)

In [ ]:
from IPython.display import display, Markdown
from llama_index.core import Document
from PIL import Image
import os
import pytesseract

# 문서의 형식 = 텍스트 
# tesseract 를 사용하기 위해서는 따로 설치를 하여야 합니다
# 튜토리얼 : https://www.allmyuniverse.com/implementing-python-ocr-with-tesseract/ 
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

image_path = r'data_png\dwld.png'
doc2_text = pytesseract.image_to_string(Image.open(image_path), lang='kor')

# 객체로 변환하기 
doc2 = Document(text=doc2_text)
# Document 객체 출력
print(doc2)

### pdf 불러오기

In [ ]:
from llama_index.core import SimpleDirectoryReader

input_dir = r"data_pdf"
reader = SimpleDirectoryReader(input_dir=input_dir)
doc1 = reader.load_data()

# doc1 = SimpleDirectoryReader(input_files=["data"]).load_data()

[Document(id_='96e0c1fc-b31e-4004-b187-ab2b86c80b92', embedding=None, metadata={'page_label': '1', 'file_name': '항만법.pdf', 'file_path': 'f:\\Project\\haemulhaemul\\data_pdf\\항만법.pdf', 'file_type': 'application/pdf', 'file_size': 270149, 'creation_date': '2024-08-31', 'last_modified_date': '2024-08-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='법제처                                                            1                                                       국가법령정보센터\n항만법 \n항만법 \n[시행 2024. 4. 25.] [법률 제19778호, 2023. 10. 24., 일부개정] \n \n       제1장 총칙\n             제1조(목적)\n             제2조(정의)\n             제3조(항만의 구분 및 명칭ㆍ위치ㆍ구역 등)\n             제4조(중앙항만정책심의회 등)\n       제2장 항만기본계획\n             제5조(항만기본계획의 수립)\n             제6조(항만기본계획의 내용)\n     

### pdf OCR로 텍스트 변환
https://medium.com/@dr.booma19/extracting-text-from-pdf-files-using-ocr-a-step-by-step-guide-with-python-code-becf221529ef

### 파일 결합

In [ ]:
all_docs = doc1 + [doc2] 
print(all_docs)

[Document(id_='9e1056d7-e2b2-48de-ba52-33c802aa0d25', embedding=None, metadata={'page_label': '1', 'file_name': '항만법.pdf', 'file_path': 'f:\\Project\\haemulhaemul\\data_pdf\\항만법.pdf', 'file_type': 'application/pdf', 'file_size': 270149, 'creation_date': '2024-08-31', 'last_modified_date': '2024-08-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='법제처                                                            1                                                       국가법령정보센터\n항만법 \n항만법 \n[시행 2024. 4. 25.] [법률 제19778호, 2023. 10. 24., 일부개정] \n \n       제1장 총칙\n             제1조(목적)\n             제2조(정의)\n             제3조(항만의 구분 및 명칭ㆍ위치ㆍ구역 등)\n             제4조(중앙항만정책심의회 등)\n       제2장 항만기본계획\n             제5조(항만기본계획의 수립)\n             제6조(항만기본계획의 내용)\n     

### 임베딩 다운로드

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model_ko = HuggingFaceEmbedding(model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr") 

### llama index 설정

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
from llama_index.core import ServiceContext

llm = Gemini(model_name='models/gemini-1.5-flash', request_timeout=120.0)

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=800, chunk_overlap=20, embed_model=embed_model_ko)
index = VectorStoreIndex.from_documents(doc1,service_context=service_context,show_progress=True)

index.storage_context.persist()

query_engine = index.as_query_engine()


### 질문과 답변 출력

In [ ]:
response = query_engine.query("항만법에 대해 말해봐")

print(response)

response = query_engine.query("해운법에 대해 말해봐")

print(response)

항만법은 항만의 지정, 개발, 관리 및 사용에 관한 사항을 규정하여 항만개발사업을 촉진하고 항만을 효율적으로 관리ㆍ운영하여 국민경제 발전에 이바지하는 것을 목적으로 합니다. 

해운법은 해상운송의 질서를 유지하고 공정한 경쟁을 이루도록 하며, 해운업의 건전한 발전과 여객 및 화물의 원활하고 안전한 운송을 도모하여 이용자의 편의를 향상시키고 국민경제 발전과 공공복리 증진에 이바지하는 것을 목적으로 합니다. 



### 멀티턴 질문과 답변

In [ ]:
# 멀티 턴 대화를 위한 history 리스트
history = []

def ask_query(query, history):
    # 대화 히스토리에 현재 쿼리를 추가
    history.append({"role": "user", "content": query})
    
    # 대화 히스토리를 문자열로 병합
    formatted_history = "\n".join([f"{item['role']}: {item['content']}" for item in history])
    
    # 쿼리 엔진에 현재 히스토리를 전달하여 응답 생성
    response = query_engine.query(formatted_history)
    
    # 응답을 히스토리에 추가
    history.append({"role": "model", "content": response})
    
    return response

# 예시 대화
query1 = "해운법에 대해 설명"
response1 = ask_query(query1, history)
print(f"Model: {response1}")

query2 = "더 자세히 알려줘"
response2 = ask_query(query2, history)
print(f"Model: {response2}")

query3 = "칸예 웨스트에 대해 설명해줘"
response3 = ask_query(query3, history)
print(f"Model: {response3}")

User: 해운법에 대해 설명
Model: 해운법은 해상운송의 질서를 유지하고 공정한 경쟁을 이루도록 하며, 해운업의 건전한 발전과 여객 및 화물의 원활하고 안전한 운송을 도모하여 이용자의 편의를 향상시키고 국민경제 발전과 공공복리 증진에 이바지하는 것을 목적으로 합니다. 

User: 더 자세히 알려줘
Model: 해운법은 해상운송의 질서를 유지하고 공정한 경쟁을 이루도록 하며, 해운업의 건전한 발전과 여객 및 화물의 원활하고 안전한 운송을 도모하여 이용자의 편의를 향상시키고 국민경제 발전과 공공복리 증진에 이바지하는 것을 목적으로 합니다. 해운법은 해운업의 종류, 해운업체의 등록 및 허가, 운임 및 운송 계약, 선박의 안전 및 환경 보호, 해운사고의 처리 등 해운업과 관련된 다양한 사항을 규정하고 있습니다. 

User: 칸예 웨스트에 대해 설명해줘
Model: 죄송합니다. 칸예 웨스트에 대한 정보는 제공할 수 없습니다. 저는 해운법에 대한 정보만 제공할 수 있습니다. 

